<h2>Лабораторная работа №4 (Дополнительная часть) по курсу "Проектирование интеллектуальных систем"</h2>

<p><b>Выполнил:</b> Саврасов П.А. группа ИУ5-24М</p>

<h3>Задание</h3>
<p>Модифицировать программный код лабораторной №3 с добавлением сохранения модели и сохранения
сводных статистик для визуализации Tensorboard.</p>

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.python.keras.datasets import cifar10

In [49]:
(xTrain, yTrain), (xTest, yTest) = cifar10.load_data()
print(xTrain.shape, yTrain.shape)
print(xTest.shape, yTest.shape)

(50000, 32, 32, 3) (50000, 1)
(10000, 32, 32, 3) (10000, 1)


In [85]:
model = keras.Sequential(
    [
        layers.InputLayer(input_shape=(32, 32, 3)),
        layers.Conv2D(32, 3, activation='relu'),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, 3, activation='relu'),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(128, 3, activation='relu'),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(512, activation='relu'),
        layers.Dense(10)
    ]
)

epochsNum = 5
optimizer = keras.optimizers.Adam()
lossFn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metricAcc = keras.metrics.SparseCategoricalAccuracy()

trainWriter = tf.summary.create_file_writer('log/train/')
testWriter = tf.summary.create_file_writer('log/test/')
trainStep = testStep = 0
batchSize = 100

Прежде всего необходимо переписать цикл обучения со встроенного на расширенный

In [94]:
for epoch in range(epochsNum):
    print(f'Epoch № {epoch} ------------------------------------')
    
    # Training loop
    for i in range(len(xTrain) // batchSize):
        xBatch = xTrain[i * batchSize : (i + 1) * batchSize]
        yBatch = yTrain[i * batchSize : (i + 1) * batchSize]
        with tf.GradientTape() as tape:
            yPred = model(xBatch, training=True)
            loss = lossFn(yBatch, yPred)
            
        gradients = tape.gradient(loss, model.trainable_weights)
        optimizer.apply_gradients(zip(gradients, model.trainable_weights))
        metricAcc.update_state(yBatch, yPred)
    
    with trainWriter.as_default():
        tf.summary.scalar("Loss", loss, step=epoch)
        tf.summary.scalar('Accuracy', metricAcc.result(), step=epoch)
    
    print('Train Accuracy: ', metricAcc.result())
    metricAcc.reset_states()
    
    # Test loop
    for i in range(len(xTest) // batchSize):
        xBatch = xTrain[i * batchSize : (i + 1) * batchSize]
        yBatch = yTrain[i * batchSize : (i + 1) * batchSize]
        yPred = model(xBatch, training=False)
        metricAcc.update_state(yBatch, yPred)

    with testWriter.as_default():
        tf.summary.scalar("Loss", loss, step=epoch)
        tf.summary.scalar('Accuracy', metricAcc.result(), step=epoch)

            
    print('Test Accuracy: ', metricAcc.result(), '\n')
    metricAcc.reset_states()

    

Epoch № 0 ------------------------------------
Train Accuracy:  tf.Tensor(0.851138, shape=(), dtype=float32)
Test Accuracy:  tf.Tensor(0.8295, shape=(), dtype=float32) 

Epoch № 1 ------------------------------------
Train Accuracy:  tf.Tensor(0.86364, shape=(), dtype=float32)
Test Accuracy:  tf.Tensor(0.8315, shape=(), dtype=float32) 

Epoch № 2 ------------------------------------
Train Accuracy:  tf.Tensor(0.8719, shape=(), dtype=float32)
Test Accuracy:  tf.Tensor(0.849, shape=(), dtype=float32) 

Epoch № 3 ------------------------------------
Train Accuracy:  tf.Tensor(0.87694, shape=(), dtype=float32)
Test Accuracy:  tf.Tensor(0.8432, shape=(), dtype=float32) 

Epoch № 4 ------------------------------------
Train Accuracy:  tf.Tensor(0.88502, shape=(), dtype=float32)
Test Accuracy:  tf.Tensor(0.835, shape=(), dtype=float32) 

